In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv('train.csv', index_col=0)
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [4]:
# Create a function to evaluate missing in data, check unique in object (categorical col)
def check_nan(df):

    check = []

    for col in df.columns:

        if df.dtypes[col] == 'object':
            type = 'Object'
            unique_obj = df[col].unique()
        else:
            type = 'Numeric'
            unique_obj = ''

        total_nan = df[col].isnull().sum()
        percent_nan = total_nan/len(df[col])
        check.append([col, type, total_nan, percent_nan, unique_obj])

    check_df = pd.DataFrame(check, columns=['Col', 'Type', 'Total_NaN', 'Percent', 'Unique'])

    return check_df

In [5]:
check = check_nan(df)
check

,Col,Type,Total_NaN,Percent,Unique
0,MSSubClass,Numeric,0,0.000000,
1,MSZoning,Object,0,0.000000,"[RL, RM, C (all), FV, RH]"
2,LotFrontage,Numeric,259,0.177397,
3,LotArea,Numeric,0,0.000000,
4,Street,Object,0,0.000000,"[Pave, Grvl]"
...,...,...,...,...,...
75,MoSold,Numeric,0,0.000000,
76,YrSold,Numeric,0,0.000000,
77,SaleType,Object,0,0.000000,"[WD, New, COD, ConLD, ConLI, CWD, ConLw, Con, ..."
78,SaleCondition,Object,0,0.000000,"[Normal, Abnorml, Partial, AdjLand, Alloca, Fa..."


In [6]:
# Check Type Object + Total_NaN == 0, to see Unique, if have any incorrect
re_check_obj = check[(check['Type'] == 'Object') & (check['Total_NaN'] == 0)]['Unique']
for i in range(len(re_check_obj)):
    print(re_check_obj.iloc[i])
# Check to see object like '?', 'Unknown', 'Dont know', 'Not Available' ... 
# It seem re_check_obj clean

['RL' 'RM' 'C (all)' 'FV' 'RH']
['Pave' 'Grvl']
['Reg' 'IR1' 'IR2' 'IR3']
['Lvl' 'Bnk' 'Low' 'HLS']
['AllPub' 'NoSeWa']
['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
['Gtl' 'Mod' 'Sev']
['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
['VinylSd' 'MetalSd' 'Wd Sdng' 'HdBoard' 'BrkFace' 'WdShing' 'CemntBd'
 'Plywood' 'AsbShng' 'Stucco' 'BrkComm' 'AsphShn' 'Stone' 'ImStucc'
 'CBlock']
['VinylSd' 'MetalSd' 'Wd Shng' 'HdBoard' 'Plywood

In [7]:
# See miss_df
miss_df = check[check['Total_NaN'] != 0]
miss_df
# <20% percent of missing seem can use imputation, drop all col > 20% misssing

,Col,Type,Total_NaN,Percent,Unique
2,LotFrontage,Numeric,259,0.177397,
5,Alley,Object,1369,0.937671,"[nan, Grvl, Pave]"
24,MasVnrType,Object,8,0.005479,"[BrkFace, None, Stone, BrkCmn, nan]"
25,MasVnrArea,Numeric,8,0.005479,
29,BsmtQual,Object,37,0.025342,"[Gd, TA, Ex, nan, Fa]"
30,BsmtCond,Object,37,0.025342,"[TA, Gd, nan, Fa, Po]"
31,BsmtExposure,Object,38,0.026027,"[No, Gd, Mn, Av, nan]"
32,BsmtFinType1,Object,37,0.025342,"[GLQ, ALQ, Unf, Rec, BLQ, nan, LwQ]"
34,BsmtFinType2,Object,38,0.026027,"[Unf, BLQ, nan, ALQ, Rec, LwQ, GLQ]"
41,Electrical,Object,1,0.000685,"[SBrkr, FuseF, FuseA, FuseP, Mix, nan]"


In [8]:
col_drop = miss_df[miss_df['Percent'] > 0.2]['Col']
df.drop(columns=col_drop.tolist(), inplace=True)
# update miss_df, result
check.drop(col_drop.index, inplace=True)
miss_df.drop(col_drop.index, inplace=True)
miss_df

,Col,Type,Total_NaN,Percent,Unique
2,LotFrontage,Numeric,259,0.177397,
24,MasVnrType,Object,8,0.005479,"[BrkFace, None, Stone, BrkCmn, nan]"
25,MasVnrArea,Numeric,8,0.005479,
29,BsmtQual,Object,37,0.025342,"[Gd, TA, Ex, nan, Fa]"
30,BsmtCond,Object,37,0.025342,"[TA, Gd, nan, Fa, Po]"
31,BsmtExposure,Object,38,0.026027,"[No, Gd, Mn, Av, nan]"
32,BsmtFinType1,Object,37,0.025342,"[GLQ, ALQ, Unf, Rec, BLQ, nan, LwQ]"
34,BsmtFinType2,Object,38,0.026027,"[Unf, BLQ, nan, ALQ, Rec, LwQ, GLQ]"
41,Electrical,Object,1,0.000685,"[SBrkr, FuseF, FuseA, FuseP, Mix, nan]"
57,GarageType,Object,81,0.055479,"[Attchd, Detchd, BuiltIn, CarPort, nan, Basmen..."


In [10]:
def convert_to_ordinal(data, col, encoder):
    encoder[col] = OrdinalEncoder()
    df_nonnull = np.array(data[col].dropna())
    impute_ordinal = encoder[col].fit_transform(df_nonnull.reshape(-1,1))
    data[col].loc[data[col].notnull()] = np.squeeze(impute_ordinal)

list_categoric = check[check['Type'] == 'Object']['Col'].tolist()

df_encode = df.copy()
encoder_dict = {}
for col in list_categoric:
    convert_to_ordinal(df_encode, col, encoder_dict)
df_encode

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,3.0,65.0,8450,1.0,3.0,3.0,0.0,4.0,0.0,...,0,0,0,0,0,2,2008,8.0,4.0,208500
2,20,3.0,80.0,9600,1.0,3.0,3.0,0.0,2.0,0.0,...,0,0,0,0,0,5,2007,8.0,4.0,181500
3,60,3.0,68.0,11250,1.0,0.0,3.0,0.0,4.0,0.0,...,0,0,0,0,0,9,2008,8.0,4.0,223500
4,70,3.0,60.0,9550,1.0,0.0,3.0,0.0,0.0,0.0,...,272,0,0,0,0,2,2006,8.0,0.0,140000
5,60,3.0,84.0,14260,1.0,0.0,3.0,0.0,2.0,0.0,...,0,0,0,0,0,12,2008,8.0,4.0,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,3.0,62.0,7917,1.0,3.0,3.0,0.0,4.0,0.0,...,0,0,0,0,0,8,2007,8.0,4.0,175000
1457,20,3.0,85.0,13175,1.0,3.0,3.0,0.0,4.0,0.0,...,0,0,0,0,0,2,2010,8.0,4.0,210000
1458,70,3.0,66.0,9042,1.0,3.0,3.0,0.0,4.0,0.0,...,0,0,0,0,2500,5,2010,8.0,4.0,266500


In [12]:
imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(df_encode)
imputed_df = pd.DataFrame(np.round(imputed), columns=df_encode.columns)
imputed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   float64
 1   MSZoning       1460 non-null   float64
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   float64
 4   Street         1460 non-null   float64
 5   LotShape       1460 non-null   float64
 6   LandContour    1460 non-null   float64
 7   Utilities      1460 non-null   float64
 8   LotConfig      1460 non-null   float64
 9   LandSlope      1460 non-null   float64
 10  Neighborhood   1460 non-null   float64
 11  Condition1     1460 non-null   float64
 12  Condition2     1460 non-null   float64
 13  BldgType       1460 non-null   float64
 14  HouseStyle     1460 non-null   float64
 15  OverallQual    1460 non-null   float64
 16  OverallCond    1460 non-null   float64
 17  YearBuilt      1460 non-null   float64
 18  YearRemo

In [13]:
imputed_df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60.0,3.0,65.0,8450.0,1.0,3.0,3.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,8.0,4.0,208500.0
1,20.0,3.0,80.0,9600.0,1.0,3.0,3.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,8.0,4.0,181500.0
2,60.0,3.0,68.0,11250.0,1.0,0.0,3.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,8.0,4.0,223500.0
3,70.0,3.0,60.0,9550.0,1.0,0.0,3.0,0.0,0.0,0.0,...,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,8.0,0.0,140000.0
4,60.0,3.0,84.0,14260.0,1.0,0.0,3.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,8.0,4.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,3.0,62.0,7917.0,1.0,3.0,3.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,8.0,4.0,175000.0
1456,20.0,3.0,85.0,13175.0,1.0,3.0,3.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,8.0,4.0,210000.0
1457,70.0,3.0,66.0,9042.0,1.0,3.0,3.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0,8.0,4.0,266500.0
1458,20.0,3.0,68.0,9717.0,1.0,3.0,3.0,0.0,4.0,0.0,...,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,8.0,4.0,142125.0


In [14]:
df_final = imputed_df.copy()
for col in list_categoric:
    df_final[col] = encoder_dict[col].inverse_transform(imputed_df[col].values.reshape(-1,1))
df_final

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60.0,RL,65.0,8450.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,2.0,2008.0,WD,Normal,208500.0
1,20.0,RL,80.0,9600.0,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,WD,Normal,181500.0
2,60.0,RL,68.0,11250.0,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,WD,Normal,223500.0
3,70.0,RL,60.0,9550.0,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,WD,Abnorml,140000.0
4,60.0,RL,84.0,14260.0,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60.0,RL,62.0,7917.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,WD,Normal,175000.0
1456,20.0,RL,85.0,13175.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,WD,Normal,210000.0
1457,70.0,RL,66.0,9042.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0,WD,Normal,266500.0
1458,20.0,RL,68.0,9717.0,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,WD,Normal,142125.0


In [15]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   float64
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   float64
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   float64
 16  OverallCond    1460 non-null   float64
 17  YearBuilt      1460 non-null   float64
 18  YearRemo